# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [1]:
import numpy as np

stateSpace = []
discreteSpace = np.arange(16)

for i in range(1,5):
    for j in range(1,5):
        stateSpace.append(str(i) + "W" + str(j) + "H")

print("State Space:\n", stateSpace)

actionSpace = ["U", "D", "R", "L", "S"]

print("\nFor the Action Space we only consider the Wolf actions, since we didn't consider the Hare to be an agent\nAction Space:", actionSpace)

successWolf = 0.8
successHare = 0.4

#functions for the transition matrices
def hareTransitionProb(s1, s2):
    s1H = s1[2]
    s2H = s2[2]
    
    if (int(s1H) + int(s2H) == 5):
        return 0
    
    if (s1H == s2H):
        return 1 - successHare
    else:
        return successHare/2
    
def wolfTransitionProb (s1, s2, a):    
    s1W = s1[0]
    s2W = s2[0]

    if (a == 'U' or a == 'D'):
        if (int(s1W) + int(s2W) == 4 or int(s1W) + int(s2W) == 6):
            return successWolf
        elif (int(s1W) + int(s2W) == int(s1W) * 2):
            return 1 - successWolf
        else:
            return 0

    if (a == 'R' or a == 'L'):
        if (int(s1W) + int(s2W) == 3 or int(s1W) + int(s2W) == 7):
            return successWolf
        elif (int(s1W) + int(s2W) == int(s1W) * 2):
            return 1 - successWolf
        else:
            return 0

    if (int(s1W) + int(s2W) == int(s1W) * 2):
        return 1
    
    return 0

#functions for the costMatrix
def determineCost(s, a):
    sW = s[0]
    sH = s[2]
    
    difference = abs(int(sW) - int(sH))
    
    #diagonal
    if (int(sW) + int(sH) == 5):
        if (a == 'S'):
            return 1
        else:
            return 0.5
    
    #vertical
    if (difference == 2):
        if (a == 'U' or a == 'D'):
            return 0
        elif (a == 'S'):
            return 0.5
        else:
            return 1
    
    #horizontal
    if (difference == 1):
        if (a == 'R' or a == 'L'):
            return 0
        elif (a == 'S'):
            return 0.5
        else:
            return 1
    
    #in the same cell
    if (a == 'S'):
        return 0
    else:
        return 1

probMatrixUp    = np.zeros((16,16))
probMatrixRight = np.zeros((16,16))
probMatrixStay  = np.zeros((16,16))

#Up action
for i in range(16):
    for j in range(16):
        probMatrixUp[i, j] = wolfTransitionProb(stateSpace[i], stateSpace[j], 'U') * \
                             hareTransitionProb(stateSpace[i], stateSpace[j])
            
        probMatrixRight[i, j] = wolfTransitionProb(stateSpace[i], stateSpace[j], 'R') * \
                             hareTransitionProb(stateSpace[i], stateSpace[j])
            
        probMatrixStay[i, j] = wolfTransitionProb(stateSpace[i], stateSpace[j], 'S') * \
                             hareTransitionProb(stateSpace[i], stateSpace[j])

probMatrixDown = probMatrixUp
probMatrixLeft = probMatrixRight

#--COST MATRIX--
# action - column matching: U, D, R, L, S
costMatrix = np.zeros((16,5))        
        
for i in range(16):
    for j in range (5):
        costMatrix[i, j] = determineCost(stateSpace[i], actionSpace[j])
        
print("\nProbability Matrix for action UP:\n", probMatrixUp)
print("\nProbability Matrix for action DOWN:\n", probMatrixDown)
print("\nProbability Matrix for action Right:\n", probMatrixRight)
print("\nProbability Matrix for action LEFT:\n", probMatrixLeft)
print("\nProbability Matrix for action STAY:\n", probMatrixStay)

print("\nCost Function for the MDP: \n", costMatrix)

State Space:
 ['1W1H', '1W2H', '1W3H', '1W4H', '2W1H', '2W2H', '2W3H', '2W4H', '3W1H', '3W2H', '3W3H', '3W4H', '4W1H', '4W2H', '4W3H', '4W4H']

For the Action Space we only consider the Wolf actions, since we didn't consider the Hare to be an agent
Action Space: ['U', 'D', 'R', 'L', 'S']

Probability Matrix for action UP:
 [[ 0.12  0.04  0.04  0.    0.    0.    0.    0.    0.48  0.16  0.16  0.    0.
   0.    0.    0.  ]
 [ 0.04  0.12  0.    0.04  0.    0.    0.    0.    0.16  0.48  0.    0.16
   0.    0.    0.    0.  ]
 [ 0.04  0.    0.12  0.04  0.    0.    0.    0.    0.16  0.    0.48  0.16
   0.    0.    0.    0.  ]
 [ 0.    0.04  0.04  0.12  0.    0.    0.    0.    0.    0.16  0.16  0.48
   0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.48  0.16  0.16  0.    0.    0.    0.    0.
   0.48  0.16  0.16  0.  ]
 [ 0.    0.    0.    0.    0.16  0.48  0.    0.16  0.    0.    0.    0.
   0.16  0.48  0.    0.16]
 [ 0.    0.    0.    0.    0.16  0.    0.48  0.16  0.    0.    0.    0.
   

### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [2]:
policy = np.zeros((16,5))

def determinePolicy(s, a):
    sW = s[0]
    sH = s[2]
    
    difference = abs(int(sW) - int(sH))
    
    #diagonal
    if (int(sW) + int(sH) == 5):
        if (a == 'S'):
            return 0
        else:
            return 0.25
    
    #vertical
    if (difference == 2):
        if (a == 'U' or a == 'D'):
            return 0.5
        else:
            return 0
    
    #horizontal
    if (difference == 1):
        if (a == 'R' or a == 'L'):
            return 0.5
        else:
            return 0
    
    #in the same cell
    if (a == 'S'):
        return 1
    else:
        return 0
    
for i in range(16):
    for j in range(5):
        policy[i, j] = determinePolicy(stateSpace[i], actionSpace[j])
        
print (policy)


[[ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.    0.    0.    0.    1.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.25  0.25  0.25  0.25  0.  ]
 [ 0.5   0.5   0.    0.    0.  ]
 [ 0.    0.    0.5   0.5   0.  ]
 [ 0.    0.    0.    0.    1.  ]]


---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [5]:
gama = 0.99

costGo = np.dot( np.linalg.inv( (np.identity(16) - gama * probMatrixUp) ), costMatrix[:,1] )

print ("The costo-to-go funtion associated with the given policy:\n",costGo)

The costo-to-go funtion associated with the given policy:
 [ 62.5803122   61.80392426  62.97262204  62.6431415   61.80392426
  62.5803122   62.6431415   62.97262204  62.97262204  62.6431415
  62.5803122   61.80392426  62.6431415   62.97262204  61.80392426
  62.5803122 ]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [4]:
# Insert your code here

---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [5]:
# Insert your code here

### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [6]:
# Insert your code here